# Praktisch - setup

Hieronder vind je enkele voorbeelden van hoe je met LLM kan connecteren.
- Of, gewoon via de provider zelf (bv. OpenAI), 
- of via een hub, zoals bv. HuggingFace,
- of via een app, zoals bv. Ollama.

Soms kan je een model gebruiken via een inference provider. Het model draait dan in de cloud by de provider en jij kan het gebruiken. \
Soms kan je de parameters van het model downloaden en het model op je eigen computer draaien.

Bekijk eens wat de mogelijkheden zijn op: https://huggingface.co/docs

- Hub & Client Libraries, in het bijzonder
    - Hub
    - Hub Python Library
- Deployment & Inference
    - Inference Providers
    - ...
- Core ML Libraries
    - Transformers
    - Sentence Transformers
- Training & Optimization
- Collaboration & Extras.

## API keys ophalen

Werk je met colab? Dan moet je eerst de volgende regels uitvoeren om een API key op te halen: \
from google.colab import userdata \
api_key = userdata.get("YOUR_API_KEY_NAME")

Werk je met een .env file, zorg dat je die ingeladen hebt. \
import dotenv \ 
dotenv.load_dotenv() \
api_key = os.getenv('YOUR_API_KEY_NAME')

## Huggingface
Je kan via een access token connecteren met verschillende huggingface modellen. Bv. met "meta-llama/Meta-Llama-3-8B-Instruct".

Link naar de Huggingface pagina waar je de cli command kan vinden om de access token van Huggingface te gebruiken:
https://huggingface.co/docs/huggingface_hub/quick-start

Link naar de Huggingface pagina waar je kan lezen hoe je met Huggingface kan interaheren:
https://huggingface.co/docs/huggingface_hub/guides/inference

Bijvoorbeeld:
(log in via de cli: huggingface-cli login)
(geef je HuggingFace access token in)
(herstart de kernel)

Modellen die je eventueel kan gebruiken:
• Llama-3.1-8b-instruct
• Llama-3.2
• Qwen2.5-7B instruct
• Qwen3

Wanneer je Huggingface modellen downloadt, gebruik makende van de pipeline uit transformers, worden die gecashed op je computer.
Met het commmando "ls -lh ~/.cache/huggingface/hub/" kan je zien welke modellen er staan en hoeveel plaats ze innemen.

Je kan ook een "space" opzetten op "https://huggingface.co/". Hierin kan je connecteren met verschillende modellen. Je hebt er CPU's en 16G RAM ter beschikking.


## Ollama
- Website: https://ollama.com/
- Github: https://github.com/ollama/ollama
- ollama-python: https://github.com/ollama/ollama-python?utm_source=chatgpt.com


## OpenAI (Dit is een voorbeeld.)
Indien je met de OpenAI API wil connecteren, heb je een OpenAI API key nodig, en een (betalend) account bij OpenAI.

Om dit een beetje te duiden, je betaalt bij ChatGPT per 'token'. ChatGPT (en andere LLM's) genereren hun teksten in stukjes die we tokens noemen. Je betaalt zowel voor tokens in de vraag ("prompt") als in het antwoord ("completion") aan een bepaald bedrag per miljoen tokens. (ref. https://openai.com/api/pricing/)

In [1]:
# Hier wordt ollama gebruikt om een lokaal model aan te spreken.

# Om een model van ollama te gebruiken, moet je eerst ollama installeren.
# Volg de instructies op https://ollama.com/download of 
# In WSL terminal:
# ollama --version
# sudo snap install ollama
# Je dient ook een model lokaal te zetten, bv. a.d.h.v. het commando "ollama pull hf.co/unsloth/Qwen3-8B-GGUF:Q4_K_M"

from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='hf.co/unsloth/Qwen3-8B-GGUF:Q4_K_M', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])

print(response['message']['content'])

# or access fields directly from the response object
print(response.message.content)


<think>
Okay, the user asked, "Why is the sky blue?" I need to explain this in a clear and accurate way. Let me start by recalling what I know about this. The sky appears blue because of Rayleigh scattering. 

First, sunlight is white, but it's made up of different colors, each with different wavelengths. When sunlight enters Earth's atmosphere, the molecules and small particles in the air scatter the light. Shorter wavelengths like blue and violet are scattered more than longer ones like red and yellow. 

Wait, why blue and not violet? Oh, right, the sun emits more blue light than violet, and our eyes are more sensitive to blue. Also, some of the violet light is absorbed by the upper atmosphere. So the scattered blue light reaches our eyes, making the sky appear blue.

But I should mention that during sunrise or sunset, the sky turns red or orange. That's because the light has to pass through more atmosphere, so the shorter wavelengths are scattered away, leaving the longer ones. 

I 

In [ ]:
# Alternatively, you can use Huggingface transformers to access the model.
# Use a pipeline as a high-level helper
# When using transformers, the model gets downloaded to your local device.

from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen2.5-7B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# In stead of using the transformers pipeline, you can also load the model directly.

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

In [ ]:
# Met onderstaande code creëer je een openAI client en een chat completion.
import os
from openai import OpenAI

# Indien je met colab werkt, gebruik dan:
# from google.colab import userdata
# client = OpenAI(
#     api_key=userdata.get('OPENAI_API_KEY'),
# )
# Werk je met een .env file, gebruik dan:
from dotenv import load_dotenv
import os
_ = load_dotenv()
print(os.getenv('OPENAI_API_KEY')[0:5])
api_key = os.getenv("OPENAI_API_KEY")  
assert isinstance(api_key, str) and api_key.startswith("sk-"), "API key not a string"
client = OpenAI(api_key=api_key.strip())


# Je kan hiermee testen of het werkt:
"""
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)
"""